In [ ]:
import sys, os

# Add project_root to sys.path so Python can find `scripts/`
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [ ]:
from general_scripts.features_engineering import quicky_data, lag_features_indicators
from general_scripts.pipelines import price_model
from general_scripts.predict import future_price_prediction
from general_scripts.lstm import LSTMModelMultiStep, LSTMModelMultiOutput
from general_scripts.helper import CustomizedLoss
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [ ]:
scaler = StandardScaler()
model_type = LSTMModelMultiStep
criterion = CustomizedLoss()
n_lags = 25
n_forecast = 2

In [ ]:
# 📂 Load dataset
file_path_1 = "../ready_data/cleaned_vn_30_data.csv"
df_1 = pd.read_csv(file_path_1)[['Date', 'VN_30_Close']]
df_1.rename(columns={'VN_30_Close': 'VN_30_F1_Close'}, inplace=True)
df_1

In [ ]:
df_vn30f1 = pd.read_csv("../VN30F1_Formatted_Cleaned.csv")[['Date', 'Close']]
df_vn30f1['Date'] = pd.to_datetime(df_vn30f1['Date'], format='%d-%m-%y')
df_vn30f1.sort_values('Date', inplace=True)
df_vn30f1.rename(columns={'Close': 'VN_30_F1_Close'}, inplace=True)
df_vn30f1

In [ ]:
df_1 = pd.concat([df_vn30f1, df_1], ignore_index=True).iloc[:-len(df_vn30f1)]
df_1['Date'] = pd.to_datetime(df_1['Date'])

df_1.sort_values('Date', inplace=True)
df_1.reset_index(inplace=True, drop=True)
df_1

In [ ]:
df_vn30f1 = quicky_data(df_vn30f1)

In [ ]:
df_vn30f1.tail(10)

In [ ]:
for i in range(5):
    baseline = ExponentialSmoothing(
    df_vn30f1['VN_30_F1_Close'].iloc[:-n_forecast*(i+1)],
    trend='add',            # additive trend
    seasonal='add',         # additive seasonality
    seasonal_periods=261    # one business‐year ≈ 261 days
    ).fit()

    # 2) Forecast the next day (one‐step ahead):
    price_forecast = baseline.forecast(n_forecast)

    # 3) If you want to extract the trend/seasonal forecasts separately:
    fitted_components = baseline.fittedvalues  # this is price = level+trend+seasonal
    level = baseline.level                     # the “smoothed level” ≈ trend
    seasonal = baseline.season                 # the seasonal factors

    print(price_forecast)

In [ ]:
df_1 = quicky_data(df_1)
df_1.tail(10)

In [ ]:
# Testing
for i in range(5):
    baseline = ExponentialSmoothing(
    df_1['VN_30_F1_Close'].iloc[:-n_forecast*(i+1)],
    trend='add',            # additive trend
    seasonal='add',         # additive seasonality
    seasonal_periods=261    # one business‐year ≈ 261 days
    ).fit()

    # 2) Forecast the next day (one‐step ahead):
    price_forecast = baseline.forecast(n_forecast)

    # 3) If you want to extract the trend/seasonal forecasts separately:
    fitted_components = baseline.fittedvalues  # this is price = level+trend+seasonal
    level = baseline.level                     # the “smoothed level” ≈ trend
    seasonal = baseline.season                 # the seasonal factors

    print(price_forecast)

    data = df_1[['residual', 'trend', 'seasonal']]

    if i:
        data = data.iloc[:-n_forecast*i]
    # 🚀 Train the model and get the test set
    model, X_test_tensor, scaler, y_pred = price_model(data, scaler, model_type, criterion, tuning=False, train_seq_len=25, test_seq_len=n_forecast, epochs=50)

# 🔮 Generate future predictions
# future_price_prediction(X_test_tensor, data, y_pred, scaler, model, num_days=n_forecast)

In [ ]:
# 📊 Select additional VN-30_F1 statistics for prediction
data = df_1

# 🚀 Train the model and get the test set
model, X_test_tensor, scaler, y_pred = price_model(data, scaler, model_type, criterion)

# 🔮 Generate future predictions
future_price_prediction(X_test_tensor, data, y_pred, scaler, model, num_days=n_forecast)

In [ ]:
# 📊 Select additional VN-30_F1 statistics for prediction
data = lag_features_indicators(df_1[["VN_30_F1_Close", 'trend',
       'seasonal', 'residual']], ['VN_30_F1_Close'])

# 🚀 Train the model and get the test set
model, X_test_tensor, scaler, y_pred = price_model(data, scaler, model_type, criterion)

# 🔮 Generate future predictions
future_price_prediction(X_test_tensor, data, y_pred, scaler, model, num_days=n_forecast)

In [ ]:
# 📊 Select additional VN-30_F1 statistics for prediction
data = lag_features_indicators(df_1, ['VN_30_F1_Close'])

# 🚀 Train the model and get the test set
model, X_test_tensor, scaler, y_pred = price_model(data, scaler, model_type, criterion)

# 🔮 Generate future predictions
future_price_prediction(X_test_tensor, data, y_pred, scaler, model, num_days=n_forecast)

In [ ]:
# 📊 Select additional VN-30_F1 statistics for prediction
data = lag_features_indicators(df_1, df_1.columns)

# 🚀 Train the model and get the test set
model, X_test_tensor, scaler, y_pred = price_model(data, scaler, model_type, criterion)

# 🔮 Generate future predictions
future_price_prediction(X_test_tensor, data, y_pred, scaler, model, num_days=n_forecast)

In [ ]:
# 📂 Load dataset
file_path_2 = "../ready_data/vn_30_F1_external_data.csv"
df_2 = pd.read_csv(file_path_2)
df_2 = quicky_data(df_2)

In [ ]:
data = df_2

# 🚀 Train the model and get the test set
model, X_test_tensor, scaler, y_pred = price_model(data, scaler, model_type, criterion)

# 🔮 Generate future predictions
future_price_prediction(X_test_tensor, data, y_pred, scaler, model, num_days=n_forecast)

In [ ]:
data = lag_features_indicators(df_2, ['VN_30_F1_Close'])

# 🚀 Train the model and get the test set
model, X_test_tensor, scaler, y_pred = price_model(data, scaler, model_type, criterion)

# 🔮 Generate future predictions
future_price_prediction(X_test_tensor, data, y_pred, scaler, model, num_days=n_forecast)

In [ ]:
data = lag_features_indicators(df_2, df_2.columns)

# 🚀 Train the model and get the test set
model, X_test_tensor, scaler, y_pred = price_model(data, scaler, model_type, criterion)

# 🔮 Generate future predictions
future_price_prediction(X_test_tensor, data, y_pred, scaler, model, num_days=n_forecast)

In [ ]:
# 📂 Load dataset
file_path_3 = "../ready_data/vn_30_F1_merged_data.csv"
df_3 = pd.read_csv(file_path_3)
df_3 = quicky_data(df_3)

In [ ]:
data = df_3

# 🚀 Train the model and get the test set
model, X_test_tensor, scaler, y_pred = price_model(data, scaler, model_type, criterion)

# 🔮 Generate future predictions
future_price_prediction(X_test_tensor, data, y_pred, scaler, model, num_days=n_forecast)

In [ ]:
data = lag_features_indicators(df_3, ['VN_30_F1_Close'])

# 🚀 Train the model and get the test set
model, X_test_tensor, scaler, y_pred = price_model(data, scaler, model_type, criterion)

# 🔮 Generate future predictions
future_price_prediction(X_test_tensor, data, y_pred, scaler, model, num_days=n_forecast)

In [ ]:
data = lag_features_indicators(df_3, df_3.columns)

# 🚀 Train the model and get the test set
model, X_test_tensor, scaler, y_pred = price_model(data, scaler, model_type, criterion)

# 🔮 Generate future predictions
future_price_prediction(X_test_tensor, data, y_pred, scaler, model, num_days=n_forecast)

In [ ]:
data = lag_features_indicators(df_3, df_1.columns)

# 🚀 Train the model and get the test set
model, X_test_tensor, scaler, y_pred = price_model(data, scaler, model_type, criterion)

# 🔮 Generate future predictions
future_price_prediction(X_test_tensor, data, y_pred, scaler, model, num_days=n_forecast)

In [ ]:
data = lag_features_indicators(df_3, df_2.columns)

# 🚀 Train the model and get the test set
model, X_test_tensor, scaler, y_pred = price_model(data, scaler, model_type, criterion)

# 🔮 Generate future predictions
future_price_prediction(X_test_tensor, data, y_pred, scaler, model, num_days=n_forecast)